In [1]:
import quimb.tensor as qtn
import quimb as qu
from local_discriminant import load_disc_parts, generate_disc_parts
from superoperator_dmrg import gibbs_mpo,super_dmrg
import os
import numpy as np


In [2]:

h=1.5
J=1
L=4
delta_t=.08
final_t=3.2 #max integration time
tol=1e-8
a=33 #0.05 33 #0.1 #32
num_steps=int(final_t/delta_t)


In [3]:

options={}
options['split_opts']={'method':'svd','cutoff':tol,'cutoff_mode':'rel','max_bond':100}
options['tebd_opts']={'tol':tol,'order':4,'progbar':False}
options['comp_opts']={'method':'svd','cutoff':tol,'cutoff_mode':'rel'}


In [4]:

I=np.eye(2)
X=np.array([[0,1],[1,0]])
Y=np.array([[0,-1j],[1j,0]])
Z=np.array([[1,0],[0,-1]])


h2=-J*(np.kron(np.kron(Z,I),np.kron(Z,I))-np.kron(np.kron(I,Z),np.kron(I,Z)))
h1=h*(np.kron(X,I)-np.kron(I,X))
ham_tp=qtn.LocalHam1D(L,H2=h2,H1=h1,cyclic=False)
ham_tm=qtn.LocalHam1D(L,H2=-h2,H1=-h1,cyclic=False) 




In [5]:

lh2=-J*(qu.pauli('Z') & qu.pauli('I')) & (qu.pauli('Z') & qu.pauli('I'))
lh1=h*qu.pauli('X') & qu.pauli('I')
local_ham=qtn.LocalHam1D(L,H2=lh2,H1=lh1,cyclic=False) 
rho_g=gibbs_mpo(local_ham,1e-10,options['split_opts'])


In [6]:

jump_ops={'x':X,'y':Y,'z':Z}#, 'y':Y}
#initialize the density matrix

#construct the discriminant: Disc = Sc[A][.]Sc[A]^dag-Sc[D][.]-[.]Sc[D]
folder=f'../DATA/tfim_J{J}_h{h}_L{L}_{''.join(jump_ops.keys())}/'
    
if os.path.exists(folder):
    loaded_scds,loaded_scas=load_disc_parts(folder)
else:
    loaded_scds,loaded_scas=generate_disc_parts(ham_tp,ham_tm,delta_t,num_steps,jump_ops,folder)

scds=[]
for item in loaded_scds.items():
    scds+=item[1]
scas=[]
for item in loaded_scas.items():
    scas+=item[1]
lscds=[-1*scd.copy() for scd in scds]
rscds=[-1*scd.copy() for scd in scds]
lscas=[sca.copy() for sca in scas]
rscas=[sca.copy() for sca in scas]

left_mpos=lscas+lscds+[scds[0].identity()]*len(lscds)
right_mpos=rscas+[scds[0].identity()]*len(lscds)+rscds


rho0=scas[0].identity().left_canonize()
rho0=qtn.MatrixProductOperator(rho0.arrays, upper_ind_id='c{}', lower_ind_id='d{}', tags='rho',shape='lrud')


Site: x_0
sa 0 done
scd_p1 0 done
scasa 0 done
scasca 0 done
Site: y_0
sa 0 done
scd_p1 0 done
scasa 0 done
scasca 0 done
Site: z_0
sa 0 done
scd_p1 0 done
scasa 0 done
scasca 0 done
Site: x_1
sa 1 done
scd_p1 1 done
scasa 1 done
scasca 1 done
Site: y_1
sa 1 done
scd_p1 1 done
scasa 1 done
scasca 1 done
Site: z_1
sa 1 done
scd_p1 1 done
scasa 1 done
scasca 1 done


In [7]:

N=5
tol=1e-6
eval,rho=super_dmrg(rho0,left_mpos,right_mpos,rho_g,N,tol,which_eval='LA')


right_sweep:  0
50%|■■■■■■■■■■□□□□□□□□□□|
left_sweep:  0
50%|■■■■■■■■■■□□□□□□□□□□||
 -1.0697699029393881e-07

right_sweep:  1
50%|■■■■■■■■■■□□□□□□□□□□|
left_sweep:  1
50%|■■■■■■■■■■□□□□□□□□□□||
 -0.5014415724080754

right_sweep:  2
50%|■■■■■■■■■■□□□□□□□□□□|
left_sweep:  2
50%|■■■■■■■■■■□□□□□□□□□□||
 -0.5014415724080762

right_sweep:  3
50%|■■■■■■■■■■□□□□□□□□□□|
left_sweep:  3
50%|■■■■■■■■■■□□□□□□□□□□||
 -0.501441572408077

right_sweep:  4
50%|■■■■■■■■■■□□□□□□□□□□|
left_sweep:  4
50%|■■■■■■■■■■□□□□□□□□□□||
 -0.5014415724080761
